In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
from mittens import GloVe
import numpy as np

In [2]:
filename = "../data/train/train_dataset.csv"

In [3]:
data = pd.read_csv(filename, header=None, names=['text','label'])

In [4]:
temp_data = []

for line in data.text:
    if type(line)==float:
        temp_data.append(',')
    else:
        temp_data.append(line)

data.text = temp_data

In [5]:
sentences = []
sentence = []

for token in data.text:
    if token != ',':
        sentence.append(token)
    else:
        tmp = ' '.join(sentence)
        sentences.append(tmp)
        sentence = []

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
X = count_model.fit_transform(sentences)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
cooccurrence = Xc.todense()

In [7]:
word2idx_mittens = sorted(count_model.vocabulary_.items(), key=lambda x: x[1])

In [8]:
pickle.dump(word2idx_mittens, open("../data/pickle_file/word2idx_mittens.pkl", "wb"))

In [9]:
cooccurrence = np.squeeze(np.asarray(cooccurrence))
glove_model = GloVe(n=300, max_iter=5000)
embeddings = glove_model.fit(cooccurrence)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 5000: loss: 500.93341064453125

In [10]:
filename = '../model/mittens_embedding.pkl'
pickle.dump(embeddings, open(filename, "wb"))